how to create a bucket?

create storage bucket?



In [ ]:
var qs = require('querystring');
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');

// TODO: set index page

function safeName(name) {
    return name.replace(/[^a-z0-9\-\.]/ig, '-').toLowerCase();
}

function setPublic(project, bucketName) {
    var client;
    // TODO:
    return authorizeGoogle()
        .then(c => client = c)
        .then(() => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b/${qs.escape(bucketName)}/acl`,
            data: {
                entity: 'allUsers',
                role: 'READER'
            }
        }))
        .then(() => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b/${qs.escape(bucketName)}/defaultObjectAcl`,
            data: {
                entity: 'allUsers',
                role: 'READER'
            }
        }))
}

function listBuckets(project, bucketName) {
    var params = {project: project};
    if(bucketName) {
        params.prefix = bucketName;
    }
    return authorizeGoogle()
        .then(client => client.request({
            url: `https://www.googleapis.com/storage/v1/b`,
            params
        }))
        .then(response => response.data.items || [])
}

function addBucket(project, bucketName) {
    return authorizeGoogle()
        .then(client => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b`,
            params: {project: project},
            data: {
                name: bucketName,
                location: 'us-central1',
                storageClass: 'regional',
                website: {
                    mainPageSuffix: 'index.html'
                }
            }
        }))
        .then(response => response.data.name)
        .then(() => setPublic(project, bucketName))
}

function createBucket(project, bucketName) {
    bucketName = safeName(bucketName);
    return listBuckets(project, bucketName)
        .then(buckets => {
            if(buckets.length > 0) {
                console.log(`bucket ${bucketName} already exists`);
                return Promise.resolve(buckets[0].name);
            }
            return addBucket(project, bucketName)
        })
        .then(() => bucketName);
}

module.exports = createBucket;


test create bucket?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var createBucket = importer.import('create a bucket');
var project = 'spahaha-ea443';
var bucketName = 'sheet-to-web.sheet-to-web.com';

describe('adding a bucket to google storage', () => {
    it('should add a bucket', () => {
        return createBucket(project, bucketName)
            .then(bucketName => {
                assert(bucketName.length > 0, 'should have added a bucket');
            })
    })
    
});


test stream to output?


In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');
var fetchOrStream = importer.import('fetch file or stream');

// locally based utility for editing styles
function streamToOutput(fileUrl, bucketName, stream) {
    var outputPath = path.join(path.resolve(process.env.PROJECT_OUTPUT),
                               fileUrl.replace(/\?.*/ig, ''));
    mkdirpSync(path.dirname(outputPath));
    var writeStream = fs.createWriteStream(outputPath);
    return fetchOrStream(stream || fileUrl, writeStream)
        .then(() => outputPath)
}

module.exports = streamToOutput;


fetch file or stream?


In [ ]:
var util = require('util');
var http = require('http');
var https = require('https');

function fetchOrStream(stream, writeStream) {
    var promise;
    if(typeof stream == 'object') {
        promise = Promise.resolve(stream);
    } else {
        var protocol = stream.includes('https://') ? https : http;
        promise = util.promisify(protocol.get.bind(protocol))(stream)
    }
    return promise
        .then(() => new Promise((resolve, reject) => {
            stream.pipe(writeStream)
            .on("error", (err) => {
                reject(err);
            })
            .on('finish', () => {
                resolve();
            });
        }))
}

module.exports = fetchOrStream;


upload files to google cloud?


In [ ]:
var mime = require('mime-types');
var path = require('path');
var {Storage} = require("@google-cloud/storage");
var importer = require('../Core');
var createBucket = importer.import('create a bucket');
var streamToOutput = importer.import('test stream to output');
var fetchOrStream = importer.import('fetch file or stream');

var storage = new Storage({  
  projectId: process.env.GOOGLE_STORAGE_PROJECT,  
  keyFilename: process.env.GOOGLE_STORAGE_CREDENTIALS
});

function streamToGoogle(fileUrl, bucketName, stream, metadata) {
    var gcsname = (stream ? fileUrl : path.basename(fileUrl)).replace(/\?.*/ig, '');
    var writeStream = storage.bucket(bucketName).file(gcsname)
        .createWriteStream({
            metadata: Object.assign({
                contentType: mime.lookup(gcsname)
            }, metadata || {})
        })
    return createBucket(project, bucketName)
        .then(() => fetchOrStream(stream || fileUrl, writeStream))
        .then(() => `https://storage.googleapis.com/${bucketName}/${gcsname}`)
}

module.exports = streamToGoogle;



copy a file in storage bucket?


In [ ]:
var importer = require('../Core');
var authorizeGoogle = importer.import('authorize google service');
var project = 'spahaha-ea443';
var qs = require('querystring');

function copyFileBucket(bucket, file) {
    var params = {project: project};
    
    return authorizeGoogle()
        .then(client => client.request({
            method: 'POST',
            url: `https://www.googleapis.com/storage/v1/b/${bucket}/o/${qs.escape(file)}/rewriteTo/b/${bucket}/o/${qs.escape(file.replace(/\.html|\.htm/ig, ''))}`,
            params
        }))
}

module.exports = copyFileBucket;


test upload files to google storage?


In [ ]:
var path = require('path');
var assert = require('assert');
var importer = require('../Core');
var streamToGoogle = importer.import('upload files google cloud');

describe('upload google storage', () => {
    
    it('should upload a file to a bucket', () => {
        return streamToGoogle(
            'https://www.sgs.com/-/media/global/images/structural-website-images/hero-images/hero-color-palette.jpg?la=en&hash=70B51DB0FA678306B2EAF2E6C4A725BAB0D12342',
            'sheet-to-web.com')
            .then(url => {
                assert(url.length > 0, 'should have a file url');
            })
    })
    
    
})
